Memory- Remember the past history (Conversation history)

In [5]:
from sceret_key import OPENAI_API_KEY
import os

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
# print(os.getenv('OPENAI_API_KEY'))

In [6]:
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.7)
prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fency name for this."
)

chain = LLMChain(llm=llm,prompt=prompt_template_name)
chain.invoke ("Mexican")

{'cuisine': 'Mexican', 'text': '\n\n"Cantina del Sol" '}

In [7]:
name = chain.invoke('Indian')

In [8]:
print(name)

{'cuisine': 'Indian', 'text': "\n\n1. Maharaja's Palace\n2. The Spice Kingdom\n3. Tandoori Terrace\n4. Saffron House\n5. The Curry Club\n6. Masala Mansion\n7. Chai Chateau\n8. Naan Nook\n9. Bollywood Bites\n10. Aroma Avenue"}


In [12]:
type(name.memory)

AttributeError: 'dict' object has no attribute 'memory'

so no memory associated with the name as the above code has an error: AttributeError: 'dict' object has no attribute 'memory'

In [13]:
# ADD memory using ConversationBufferMemory

from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()
chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)
name = chain.invoke("Mexican")
print(name)

{'cuisine': 'Mexican', 'history': '', 'text': '\n\nCasa de Sabor (House of Flavor)'}


In [14]:
name = chain.invoke("indian")
print(name)

{'cuisine': 'indian', 'history': 'Human: Mexican\nAI: \n\nCasa de Sabor (House of Flavor)', 'text': '\n\n"Maharaja\'s Palace: A Taste of India"'}


In [15]:
print(chain.memory.buffer)

Human: Mexican
AI: 

Casa de Sabor (House of Flavor)
Human: indian
AI: 

"Maharaja's Palace: A Taste of India"




We can store memory data in the database as a shared transcript for our customers. 

**Problem**: The `ConversationBufferMemory` grows indefinitely, which means each time you ask a new question, it returns the entire conversation history. This increases the token usage, leading to higher costs from OpenAI.

**Solution**: We should limit the memory buffer size. This can be achieved by using `ConversationChain` with a configured buffer limit.

In [22]:
 from langchain.chains import ConversationChain
convo = ConversationChain(llm=OpenAI(temperature=0.7))
print(convo.prompt.template)
                

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [23]:
convo.invoke('Who won the first cricket world cup?')

{'input': 'Who won the first cricket world cup?',
 'history': '',
 'response': ' The first cricket world cup was won by the West Indies in 1975.'}

In [24]:
convo.invoke("what is 5+5")

{'input': 'what is 5+5',
 'history': 'Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was won by the West Indies in 1975.',
 'response': '  The answer is 10.'}

In [25]:
convo.invoke("who has the captain of the winning team")

{'input': 'who has the captain of the winning team',
 'history': 'Human: Who won the first cricket world cup?\nAI:  The first cricket world cup was won by the West Indies in 1975.\nHuman: what is 5+5\nAI:   The answer is 10.',
 'response': ' The captain of the winning team for the first cricket world cup in 1975 was Clive Lloyd.'}

In [26]:
print(convo.memory.buffer)

Human: Who won the first cricket world cup?
AI:  The first cricket world cup was won by the West Indies in 1975.
Human: what is 5+5
AI:   The answer is 10.
Human: who has the captain of the winning team
AI:  The captain of the winning team for the first cricket world cup in 1975 was Clive Lloyd.


In [28]:
# Restriction to window memory(specific size of data)

from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory( k=1) # k deontes how many conversation need to be remember

convo = ConversationChain(llm=OpenAI(temperature=0.7), memory=memory)
convo.invoke("Who won the first cricket world cup?")

{'input': 'Who won the first cricket world cup?',
 'history': '',
 'response': ' The first cricket world cup was won by the West Indies in 1975. They defeated Australia in the final by 17 runs. The tournament was held in England and was officially called the Prudential Cup. Did you know that the West Indies team was led by Clive Lloyd and had players like Viv Richards, Gordon Greenidge, and Michael Holding?'}

In [29]:
convo.run('what is 5+5?')

' 5+5 is equal to 10.'

In [30]:
convo.invoke("who has the captain of the winning team")

{'input': 'who has the captain of the winning team',
 'history': 'Human: what is 5+5?\nAI:  5+5 is equal to 10.',
 'response': ' I do not have enough context to accurately answer that question. Can you provide me with more information?'}

In [31]:
convo.run("who has the captain of the winning team")

' The captain of the winning team could vary depending on the specific sport or game being played. Can you specify which team and game you are referring to?'